Preparing Data

In [7]:
import csv
from dbfread import DBF

def dbf_to_csv(dbf_filename, csv_filename):
    # Open the .dbf file using dbfread
    dbf = DBF(dbf_filename, load=True)
    
    # Open a .csv file to write the data
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        
        # Write the header (field names from the .dbf file)
        writer.writerow(dbf.field_names)
        
        # Write the rows of data
        for record in dbf:
            writer.writerow(record.values())

# Example usage
dbf_filename = 'seamounts.dbf'  # Replace with the path to your .dbf file
csv_filename = 'seamounts.csv'  # Replace with your desired output .csv file name

dbf_to_csv(dbf_filename, csv_filename)


In [ ]:
import os
import requests
import pandas as pd
import math

# Parameters
csv_file = "seamounts.csv"  # Path to the .dbf file
output_folder = "seamounts_galore"  # Folder to save downloaded images
tile_pixels = 800  # Width and height of the image in pixels
tile_width_deg = 1.0  # Fixed width in degrees
api_base_url = "https://www.gmrt.org/services/ImageServer"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load CSV file
try:
    df = pd.read_csv(csv_file)
    print(f"Loaded {len(df)} records from {csv_file}.")
except Exception as e:
    print(f"Error loading .csv file: {e}")
    exit()

# Process each record
for index, row in df.iterrows():
    try:
        # Extract data from the row
        file_name = str(row["PEAKID"])  # Use PEAKID as the file name
        center_lon = float(row["LONG"])  # Longitude from LONG
        center_lat = float(row["LAT"])  # Latitude from LAT

        # Calculate the adjusted height for square tiles
        adjusted_height_deg = tile_width_deg * math.cos(math.radians(center_lat))

        # Calculate tile bounds
        minlatitude = center_lat - (adjusted_height_deg / 2)
        maxlatitude = center_lat + (adjusted_height_deg / 2)
        minlongitude = center_lon - (tile_width_deg / 2)
        maxlongitude = center_lon + (tile_width_deg / 2)

        # Debugging: Print calculated values
        print(f"PEAKID: {file_name}")
        print(f"Center Lat: {center_lat}, Center Lon: {center_lon}")
        print(f"Adjusted Height (degrees): {adjusted_height_deg}")
        print(f"Bounds: {minlatitude} to {maxlatitude}, {minlongitude} to {maxlongitude}")

        # Prepare API parameters
        params = {
            "minlatitude": minlatitude,
            "maxlatitude": maxlatitude,
            "minlongitude": minlongitude,
            "maxlongitude": maxlongitude,
            "width": tile_pixels,
            "mask": "false",
            "download": "true",
        }

        # Construct file name
        output_file = os.path.join(output_folder, f"{file_name}.png")

        # Download the image
        print(f"Downloading square image for PEAKID {file_name} at {center_lat}, {center_lon}...")
        response = requests.get(api_base_url, params=params)

        if response.status_code == 200:
            # Save the image
            with open(output_file, "wb") as file:
                file.write(response.content)
            print(f"Saved: {output_file}")
        else:
            print(f"Failed to download for PEAKID {file_name} - HTTP {response.status_code}")

    except Exception as e:
        print(f"Error processing row {index}: {e}")

print("Download complete.")


Loaded 33452 records from seamounts.csv.
PEAKID: 26000.0
Center Lat: 84.9797363281, Center Lon: 2.7625
Adjusted Height (degrees): 0.08750805927002411
Bounds: 84.93598229846499 to 85.02349035773501, 2.2625 to 3.2625
Saved: seamounts_galore/26000.0.png
PEAKID: 26157.0
Center Lat: 84.9352918837, Center Lon: 9.14305555556
Adjusted Height (degrees): 0.08828075901556476
Bounds: 84.89115150419222 to 84.97943226320778, 8.64305555556 to 9.64305555556
Saved: seamounts_galore/26157.0.png
PEAKID: 26158.0
Center Lat: 84.9380696614999, Center Lon: 9.18333333332999
Adjusted Height (degrees): 0.0882324668324533
Bounds: 84.89395342808366 to 84.98218589491613, 8.68333333332999 to 9.68333333332999
Saved: seamounts_galore/26158.0.png
PEAKID: 26228.0
Center Lat: 84.9075141059, Center Lon: 8.74861111111
Adjusted Height (degrees): 0.08876366941205345
Bounds: 84.86313227119398 to 84.95189594060604, 8.24861111111 to 9.24861111111
Saved: seamounts_galore/26228.0.png
PEAKID: 26229.0
Center Lat: 84.9130696615, Ce